In [2]:
import uuid
str(uuid.uuid4())

'f83a70a8-d5ba-44db-a15e-ba17fbaa7805'

In [7]:
from sagas.util.name_util import to_global_id, from_global_id
import io_utils

prefix='/pi/artifacts'
user='system'
folder=to_global_id('blueprint', 'demoProducts')
full_dir='%s/%s/%s'%(prefix, user, folder)
print(full_dir)
io_utils.create_dir(full_dir)

/pi/artifacts/system/Ymx1ZXByaW50OmRlbW9Qcm9kdWN0cw==


In [15]:
asset_dir='./assets/productsDomain/seed'
build_dir='./assets/productsDomain/build'

In [10]:
import os
import platform

def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime

print(creation_date(asset_dir))

1553238674.9673362


In [12]:
import os.path, time
file=asset_dir+'/IssuanceTestData.xml'
print("last modified: %s" % time.ctime(os.path.getmtime(file)))
print("created: %s" % time.ctime(os.path.getctime(file)))

last modified: Fri Mar 22 15:11:29 2019
created: Fri Mar 22 15:11:29 2019


In [14]:
import os, time
(mode, ino, dev, nlink, uid, gid, size, atime, mtime, ctime) = os.stat(file)
print("last modified: %s" % time.ctime(mtime))
print(mtime)

last modified: Fri Mar 22 15:11:29 2019
1553238689


In [29]:
file='IssuanceTestData.xml'
pytime = os.path.getmtime(os.path.join(asset_dir, file))
# ...
txttime = os.path.getmtime(os.path.join(build_dir, file))
# ...
print(pytime, txttime)
if (txttime > pytime):
    print(True)
else:
    print(False)

1553274405.6859667 1553274244.5245712
False


In [20]:
from sagas.ofbiz.services import OfService as s
target=os.path.join(asset_dir, file)
full_path=os.path.abspath(target)
ok, result=s().entityImport(filename=full_path)
print(ok,result)

True {'messages': ['Got 26 entities to write to the datasource.']}


In [25]:
import os
import xml.etree.ElementTree as ET
from sagas.ofbiz.entity_prefabs import EntityPrefabs

print(full_path)
tree = ET.parse(full_path)
root = tree.getroot()

prefabs = EntityPrefabs()
rs, ids=prefabs.convert_to_record_set(root)
# map(token, TaStringEntries)
print(rs)
# token list
print(ids)

/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/assets/productsDomain/seed/IssuanceTestData.xml
{'T3JkZXJIZWFkZXI6REVNTzgxMDE1': values {
  key: "createdBy"
  value: "admin"
}
values {
  key: "currencyUom"
  value: "USD"
}
values {
  key: "entryDate"
  value: "2011-08-12 23:17:11.507"
}
values {
  key: "grandTotal"
  value: "105.14"
}
values {
  key: "orderId"
  value: "DEMO81015"
}
values {
  key: "orderTypeId"
  value: "SALES_ORDER"
}
values {
  key: "priority"
  value: "2"
}
values {
  key: "productStoreId"
  value: "9000"
}
values {
  key: "remainingSubTotal"
  value: "79.56"
}
values {
  key: "salesChannelEnumId"
  value: "WEB_SALES_CHANNEL"
}
values {
  key: "statusId"
  value: "ORDER_APPROVED"
}
values {
  key: "visitId"
  value: "10043"
}
entityName: "OrderHeader"
, 'T3JkZXJJdGVtOkRFTU84MTAxNeKWqzAwMDAx': values {
  key: "changeByUserLoginId"
  value: "admin"
}
values {
  key: "isModifiedPrice"
  value: "N"
}
values {
  key: "isPromo"
  value: "N"
}
values {
  key: "i

In [34]:
f=os.path.join(build_dir, 'last_modify.json')
base=os.path.basename(f)
print(base)

last_modify.json


In [35]:
import io_utils
import json_utils

last_mods={}
for f in io_utils.list_files(asset_dir):
    print(os.path.getmtime(f))
    base=os.path.basename(f)
    last_mods[base]=os.path.getmtime(f)

modify_recs=os.path.join(build_dir, 'last_modify.json')
print(modify_recs)
json_utils.write_json_to_file(modify_recs, last_mods)

1553238689.0159047
1553274405.6859667
1553238689.0164716
./assets/productsDomain/build/last_modify.json


In [39]:
recs=json_utils.read_json_file(modify_recs)
for k,v in recs.items():
    print(k,v)

ProductPriceTestData.xml 1553238689.0159047
IssuanceTestData.xml 1553274405.6859667
ProductTestData.xml 1553238689.0164716


In [44]:
modified_set=[]
for f in io_utils.list_files(asset_dir):
    dt=os.path.getmtime(f)
    print(dt)
    base=os.path.basename(f)
    if base in recs:
        if dt>recs[base]:
            print('file %s modified'%base)
            modified_set.append(base)
    else:
        # is new file
        modified_set.append(base)

print('modified files:', modified_set)

1553276887.9454908
file ProductPriceTestData.xml modified
1553274405.6859667
1553238689.0164716
modified files: ['ProductPriceTestData.xml']
